In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Large_B-cell_Lymphoma/GSE182362'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"miR-155-regulated mTOR and Toll-like receptor 5 in gastric diffuse large B-cell lymphoma"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['cell line: B-cell lymphoma cell line U2932', 'tissue: cell line derived from diffuse large B-cell lymphoma'], 1: ['tissue: cell line derived from diffuse large B-cell lymphoma', 'treatment: transfected with miR-200c'], 2: ['treatment: transfected with an empty vector', 'treatment: transfected with miR-200a', 'treatment: transfected with miR-200b', nan]}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if the dataset likely contains gene expression data
is_gene_available = False  # Based on series title, it suggests miRNA data which is not suitable.

# Variable Availability and Data Type Conversion
# Based on the provided sample characteristics dictionary, there is no direct information about 'Large_B-cell_Lymphoma', 'age', or 'gender'.
trait_row = age_row = gender_row = None  # set to different values when applicable

# Since no trait, age or gender data is available, we won't define conversion functions.

# Save cohort information
save_cohort_info('GSE182362', './preprocessed/Large_B-cell_Lymphoma/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction step to be skipped as trait_row is None
